<a href="https://colab.research.google.com/github/erinjsoto/capstone_recommendation_system/blob/main/ErinSoto_Reference_Notebook_Milestone_2_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Music Recommendation System**

# **Milestone 2**

Now that we have explored the data, let's apply different algorithms to build recommendation systems.

**Note:** Use the shorter version of the data, i.e., the data after the cutoffs as used in Milestone 1.

## **Load the dataset**

In [ ]:
# %%shell
# jupyter nbconvert --to html /content/drive/MyDrive/Colab_Notebooks/spotify_music/ErinSoto_Reference_Notebook_Milestone_2_Recommendation_Systems.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab_Notebooks/spotify_music/ErinSoto_Reference_Notebook_Milestone_2_Recommendation_Systems.ipynb to html
[NbConvertApp] Writing 470604 bytes to /content/drive/MyDrive/Colab_Notebooks/spotify_music/ErinSoto_Reference_Notebook_Milestone_2_Recommendation_Systems.html


In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Used to ignore the warning given as output of the code
import warnings
warnings.filterwarnings('ignore')

# Basic libraries of python for numeric and dataframe computations
import numpy as np
import pandas as pd

# Basic library for data visualization
import matplotlib.pyplot as plt

# Slightly advanced library for data visualization
import seaborn as sns

# To compute the cosine similarity between two vectors
from sklearn.metrics.pairwise import cosine_similarity

# A dictionary output that does not raise a key error
from collections import defaultdict

# A performance metrics in sklearn
from sklearn.metrics import mean_squared_error

# To do label encoding
from sklearn.preprocessing import LabelEncoder

In [ ]:

# Load the dataset you have saved at the end of milestone 1

df_final = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/spotify_music/my_data.csv')


In [ ]:
df_final.head()

,user_id,song_id,play_count,title,release,artist_name,year,business_id
0,6958,SOBDVAK12AC90759A2,1,Daisy And Prudence,Distillation,Erin McKeown,2000,447
1,6958,SOBIMTY12A6D4F931F,1,The Ballad of Michael Valentine,Sawdust,The Killers,2004,512
2,6958,SOBKRVG12A8C133269,1,I Stand Corrected (Album),Vampire Weekend,Vampire Weekend,2007,549
3,6958,SOBUBLL12A58A795A8,1,They Might Follow You,Tiny Vipers,Tiny Vipers,2007,703
4,6958,SOBVKFF12A8C137A79,1,Monkey Man,You Know I'm No Good,Amy Winehouse,2007,719


### **Popularity-Based Recommendation Systems**

Let's take the count and sum of play counts of the songs and build the popularity recommendation systems based on the sum of play counts.

In [ ]:
# Calculating average play_count
average_count = df_final.groupby('song_id').mean()['play_count']        # Hint: Use groupby function on the song_id column

# Calculating the frequency a song is played
play_freq = df_final.groupby('song_id').count()['play_count']           # Hint: Use groupby function on the song_id column

In [ ]:
# Making a dataframe with the average_count and play_freq
final_play = pd.DataFrame({'avg_count': average_count, 'play_freq': play_freq})

# Let us see the first five records of the final_play dataset
final_play.head()

,avg_count,play_freq
song_id,,
SOABJBU12A8C13F63F,1.622642,265
SOABJTC12A58A7DE0E,1.492424,132
SOADJQJ12A8C141D38,1.729216,421
SOADQPP12A67020C82,1.728070,114
SOAFQGA12A8C1367FA,1.452174,115


Now, let's create a function to find the top n songs for a recommendation based on the average play count of song. We can also add a threshold for a minimum number of playcounts for a song to be considered for recommendation.

In [ ]:
# Build the function to find top n songs

def top_n_products(final_play, n, min_interaction):
    
    # Finding products with minimum number of interactions
    recommendations = final_play[final_play['play_freq'] > min_interaction]
    
    # Sorting values with respect to average rating 
    recommendations = recommendations.sort_values(by='avg_count', ascending=False)
    
    return recommendations.index[:n]

In [ ]:
# Recommend top 10 songs using the function defined above
list(top_n_products(final_play, 10, 50))

['SOSJSSU12A6D4F8F41',
 'SOVIZNF12AF72A710A',
 'SOQGSUC12A8C13B66D',
 'SOZVVRE12A8C143150',
 'SONYKOW12AB01849C9',
 'SOOGNOZ12AAF3B2936',
 'SOVWBYM12A6D4F8A22',
 'SOFRQTD12A81C233C0',
 'SOBRHVR12A8C133F35',
 'SOBONKR12A58A7A7E0']

### **User User Similarity-Based Collaborative Filtering**

To build the user-user-similarity-based and subsequent models we will use the "surprise" library.

In [ ]:
# Install the surprise package using pip.
!pip install surprise 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633986 sha256=863e14078297c18f428eee9d336513344f57aec2d7b83a5d0807b8b8c7793ca1
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
# Import necessary libraries

# To compute the accuracy of models
from surprise import accuracy

# This class is used to parse a file containing play_counts, data should be in structure - user; item; play_count
from surprise.reader import Reader

# Class for loading datasets
from surprise.dataset import Dataset

# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV

# For splitting the data in train and test dataset
from surprise.model_selection import train_test_split

# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic

# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD

# For implementing KFold cross-validation
from surprise.model_selection import KFold

# For implementing clustering-based recommendation system
from surprise import CoClustering

### Some useful functions

Below is the function to calculate precision@k and recall@k, RMSE and F1_Score@k to evaluate the model performance.

**Think About It:** Which metric should be used for this problem to compare different models?

In [ ]:
# The function to calulate the RMSE, precision@k, recall@k, and F_1 score
def precision_recall_at_k(model, k = 30, threshold = 1.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    
    # Making predictions on the test data
    predictions=model.test(testset)
    
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key = lambda x : x[0], reverse = True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[ : k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[ : k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set Precision to 0 when n_rec_k is 0

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set Recall to 0 when n_rel is 0

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    
    # Mean of all the predicted precisions are calculated
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)), 3)

    # Mean of all the predicted recalls are calculated
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)), 3)
    
    accuracy.rmse(predictions)

    # Command to print the overall precision
    print('Precision: ', precision)

    # Command to print the overall recall
    print('Recall: ', recall)
    
    # Formula to compute the F-1 score
    print('F_1 score: ', round((2 * precision * recall) / (precision + recall), 3))

**Think About It:** In the function precision_recall_at_k above the threshold value used is 1.5. How precision and recall are affected by changing the threshold? What is the intuition behind using the threshold value of 1.5? 

In [ ]:
# Instantiating Reader scale with expected rating scale 
reader = Reader(rating_scale = (0, 5)) #use rating scale (0, 5)

# Loading the dataset
data = Dataset.load_from_df(df_final[['user_id', 'song_id', 'play_count']], reader) # Take only "user_id","song_id", and "play_count"

# Splitting the data into train and test dataset
trainset, testset = train_test_split(data, test_size=0.4, random_state = 42) # Take test_size = 0.4

**Think About It:** How changing the test size would change the results and outputs?

In [ ]:
# Build the default user-user-similarity model
sim_options = {'name': 'cosine',
               'user_based':True}

# KNN algorithm is used to find desired similar items
sim_user_user = KNNBasic(random_state=1) # Use random_state = 1 

# Train the algorithm on the trainset, and predict play_count for the testset
sim_user_user.fit(trainset)

# Let us compute precision@k, recall@k, and f_1 score with k = 30
precision_recall_at_k(sim_user_user) # Use sim_user_user model

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0626
Precision:  0.009
Recall:  0.003
F_1 score:  0.004


💡 **Observations and Insights:**


*   F_1 score is 0.492 meaning this model could be a lot better in predicting methods
*   RMSE is 1.0626 also indicating that this model has lots of room for imporovement
*   The precision is 0.413 meaning that out of 30 recommended items 0.413 were relevant to the user 
*   The recall is 0.608 meaning that out of 30 relevant products 0.608 are recommended



In [ ]:
# Predicting play_count for a sample user with a listened song
sim_user_user.predict('6958', '1671', r_ui = 2, verbose = True) # Use user id 6958 and song_id 1671

user: 6958       item: 1671       r_ui = 2.00   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
# Predicting play_count for a sample user with a song not-listened by the user
sim_user_user.predict('6958', '3232', verbose = True) # Use user_id 6958 and song_id 3232

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})


💡 **Observations and Insights:**

*    Per the initial prediction of user 6958 and song 1671 the recommendation r_ui is 2 while my model predicted 1.6989 which proves observations above that the model is not as efficient as it could be. 
*    There is a 0.3011 discrepency between the r_ui and the user-to-user prediction
*   Per the second predictor r_ui is unknown as the song 3232 has not been listened to by user, the model predicted a 1.70 reccomendation rating


Now, let's try to tune the model and see if we can improve the model performance.

In [ ]:
# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [10, 20, 30], 'min_k': [3, 6, 9],
              'sim_options': {'name': ["cosine", 'pearson', "pearson_baseline"],
                              'user_based': [True], "min_support": [2, 4]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
gs.fit(data) # Use entire data for GridSearch

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# Using the optimal similarity measure for item-item based collaborative filtering
sim_options = {'name': 'pearson_baseline',
               'user_based': True}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_user_user_optimized = KNNBasic(sim_options=sim_options, k =30 , min_k =6, random_state = 1, verbose = True)

# Training the algorithm on the trainset
sim_user_user_optimized.fit(trainset)

# Let us compute precision@k and recall@k also with k =10
precision_recall_at_k(sim_user_user_optimized)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0534
Precision:  0.414
Recall:  0.703
F_1 score:  0.521


💡 **Observations and Insights:**

*   f_1 score went from 0.492 to 0.521 after optimization of hyperparameters 
*   After implmeenting hyperparameters precision and recall both incresed meaning that tuning the model increased efficienciy in the model
*   Tuning the model increased model performance 
*   RMSE	1.0534
*   Precision	0.414
*   Recall	0.703




In [ ]:
# Predict the play count for a user who has listened to the song. Take user_id 6958, song_id 1671 and r_ui = 2
sim_user_user_optimized.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
# Predict the play count for a song that is not listened to by the user (with user_id 6958) 
sim_user_user_optimized.predict("6958", "3232", verbose = True) # Use user_id 6958 and song_id 3232

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

💡 **Observations and Insights:**

*    Per the initial prediction of user 6958 and song 1671 the recommendation r_ui is 2 while my model predicted 1.70  
*    There is a 0.3 discrepency between the r_ui and the user-to-user prediction
*   Per the second predictor r_ui is unknown as the song 3232 has not been listened to by user, the model predicted a 1.70 reccomendation rating similiar to the user to user model prior to tuning



**Think About It:** Along with making predictions on listened and unknown songs can we get 5 nearest neighbors (most similar) to a certain song?

In [ ]:
# Use inner id 0
sim_user_user_optimized.get_neighbors(0, k = 5)

[42, 1131, 17, 186, 249]

Below we will be implementing a function where the input parameters are:

- data: A **song** dataset
- user_id: A user-id **against which we want the recommendations**
- top_n: The **number of songs we want to recommend**
- algo: The algorithm we want to use **for predicting the play_count**
- The output of the function is a **set of top_n items** recommended for the given user_id based on the given algorithm

In [ ]:
def get_recommendations(data, user_id, top_n, algo):
    
    # Creating an empty list to store the recommended product ids
    recommendations = []
    
    # Creating an user item interactions matrix 
    user_item_interactions_matrix = data.pivot_table(index = 'user_id', columns = 'song_id', values = 'play_count')
    
    # Extracting those business ids which the user_id has not visited yet
    non_interacted_products = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()
    
    # Looping through each of the business ids which user_id has not interacted yet
    for item_id in non_interacted_products:
        
        # Predicting the ratings for those non visited restaurant ids by this user
        est = algo.predict(user_id, item_id).est
        
        # Appending the predicted ratings
        recommendations.append((item_id, est))

    # Sorting the predicted ratings in descending order
    recommendations.sort(key = lambda x : x[1], reverse = True)

    return recommendations[:top_n] # Returing top n highest predicted rating products for this user

In [ ]:
# Make top 5 recommendations for user_id 6958 with a similarity-based recommendation engine
recommendations = get_recommendations(df_final, 6958, 5, sim_user_user_optimized)

In [ ]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_ratings"
pd.DataFrame(recommendations, columns = ['song_id', 'predicted_ratings'])

,song_id,predicted_ratings
0,SOSJSSU12A6D4F8F41,3.672605
1,SOQGOPT12AAF3B2B27,2.571082
2,SONYKOW12AB01849C9,2.553335
3,SOAUWYT12A81C206F1,2.518269
4,SOMDVSL12A6D4F7230,2.498731


💡 **Observations and Insights:**


*    After adding input paramaters I was able to compute a list of song_ids with their predicted ratings
*   This is still using a user-user, so this is comparing data from similiar users listened to songs and provided recommendations based on this

### Correcting the play_counts and Ranking the above songs

In [ ]:
def ranking_songs(recommendations, final_rating):
  # Sort the songs based on play counts
  ranked_songs = final_rating.loc[[items[0] for items in recommendations]].sort_values('play_freq', ascending = False)[['play_freq']].reset_index()

  # Merge with the recommended songs to get predicted play_count
  ranked_songs = ranked_songs.merge(pd.DataFrame(recommendations, columns = ['song_id', 'predicted_ratings']), on = 'song_id', how = 'inner')

  # Rank the songs based on corrected play_counts
  ranked_songs['corrected_ratings'] = ranked_songs['predicted_ratings'] - 1 / np.sqrt(ranked_songs['play_freq'])

  # Sort the songs based on corrected play_counts
  ranked_songs = ranked_songs.sort_values('corrected_ratings', ascending = False)
  
  return ranked_songs
  print()

**Think About It:** In the above function to correct the predicted play_count a quantity 1/np.sqrt(n) is subtracted. What is the intuition behind it? Is it also possible to add this quantity instead of subtracting?

In [ ]:
# Applying the ranking_songs function on the final_play data
ranking_songs(recommendations, final_play)

,song_id,play_freq,predicted_ratings,corrected_ratings
3,SOSJSSU12A6D4F8F41,107,3.672605,3.575932
0,SONYKOW12AB01849C9,618,2.553335,2.513109
2,SOQGOPT12AAF3B2B27,109,2.571082,2.475299
1,SOAUWYT12A81C206F1,411,2.518269,2.468943
4,SOMDVSL12A6D4F7230,97,2.498731,2.397196


💡 **Observations and Insights:**

*   Predicted ratings in this model are higher than the actual rating
*    Its interesting to note that these song_ids are supposed to be the top n songs. Howevever, I argue that the actual ratings of theses songs are lower than what I would expect. I originally thought that users would have top songs that they play over and over but as you notice song _id SONYKOW12AB01849C9 is played roughly 618 times with a predicted rating of 2.55 and actual rating of 2.51




### Item Item Similarity-based collaborative filtering recommendation systems 

In [ ]:
# Apply the item-item similarity collaborative filtering model with random_state = 1 and evaluate the model performance
# Declaring the similarity options
sim_options = {'name': 'cosine',
               'user_based': False}

# KNN algorithm is used to find desired similar items
sim_item_item = KNNBasic(sim_options = sim_options, random_state = 1, verbose = True)

# Train the algorithm on the trainset, and predict ratings for the test set
sim_item_item.fit(trainset)

# Let us compute precision@k, recall@k, and f_1 score with k = 10
precision_recall_at_k(sim_item_item)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0394
Precision:  0.307
Recall:  0.562
F_1 score:  0.397


💡 **Observations and Insights:**

*   Root mean square error (RMSE) is 0.9576 meaning our algorithm worked but could be better. 
*   The precision is 0.307 meaning that out of the recommended items 0.307 were relevant to the user. 
*   The recall is 0.562 meaning that out of the relevant products 0.562 products are recommended.
*  With an F_1 score of 0.397 which is representative of both recall and precision.
*  Based on the F_1 score this model is not great

In [ ]:
# Predicting play count for a sample user_id 6958 and song (with song_id 1671) heard by the user
sim_item_item.predict("6958", "1671", r_ui =2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
# Predict the play count for a user that has not listened to the song (with song_id 1671)
sim_item_item.predict("6958", "3232",  verbose = True) # Use user_id 6958 and song_id 3232

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

💡 **Observations and Insights:**

*    The predicted rate compared to r_ui is 1.70 this is the same predicted rate as song_id 3232 for the same user  

In [ ]:
# Apply grid search for enhancing model performance

# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [10, 20, 30], 'min_k': [3, 6, 9],
              'sim_options': {'name': ["cosine", 'pearson', "pearson_baseline"],
                              'user_based': [False], "min_support": [2, 4]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
gs.fit(data)

# Find the best RMSE score
print(gs.best_score['rmse'])

# Extract the combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

**Think About It:** How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the list of hyperparameters [here](https://surprise.readthedocs.io/en/stable/knn_inspired.html).

In [ ]:
# Apply the best modle found in the grid search
# Using the optimal similarity measure for item-item based collaborative filtering
sim_options = {'name': 'msd', 'user_based': False}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_item_item_optimized = KNNBasic(sim_options=sim_options, k =30 , min_k =6, random_state = 1, verbose = True)

# Training the algorithm on the trainset
sim_item_item_optimized.fit(trainset)

# Let us compute precision@k and recall@k, f1_score and RMSE
precision_recall_at_k(sim_item_item_optimized)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0423
Precision:  0.34
Recall:  0.563
F_1 score:  0.424


💡 **Observations and Insights:**


*   Root mean square error (RMSE) is 1.0423 
*   The precision is 0.34 meaning that out of the recommended items 0.34 were relevant to the user. 
*   The recall is 0.563 meaning that out of the relevant products 0.563 products are recommended.
*  With an F_1 score of 0.424 which is representative of both recall and precision.

In [ ]:
# Predict the play_count by a user(user_id 6958) for the song (song_id 1671)
sim_item_item_optimized.predict("6958", "1671", verbose = True)

user: 6958       item: 1671       r_ui = None   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=None, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
# Predicting play count for a sample user_id 6958 with song_id 3232 which is not heard by the user
sim_item_item_optimized.predict("6958", "3232", verbose = True)

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=1.6989607635206787, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
# Find five most similar items to the item with inner id 0
sim_item_item_optimized.get_neighbors(0, k = 5)

[3, 10, 24, 30, 36]

In [ ]:
# Making top 5 recommendations for user_id 6958 with item_item_similarity-based recommendation engine
recommendations = get_recommendations(df_final, 6958, 5, sim_item_item_optimized)

In [ ]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_play_count"
pd.DataFrame(recommendations, columns = ['song_id', 'predicted_play_count'])

,song_id,predicted_play_count
0,SOZVVRE12A8C143150,2.048219
1,SOHEMBB12A6701E907,1.949116
2,SOHRQFL12A6D4F3C8A,1.885211
3,SOMLYJD12A58A7B155,1.771190
4,SOEHWGF12A6D4F8B2B,1.743513


In [ ]:
# Applying the ranking_songs function
ranking_songs(recommendations, final_play)

,song_id,play_freq,predicted_ratings,corrected_ratings
3,SOZVVRE12A8C143150,150,2.048219,1.966570
2,SOHEMBB12A6701E907,232,1.949116,1.883463
1,SOHRQFL12A6D4F3C8A,233,1.885211,1.819698
4,SOMLYJD12A58A7B155,133,1.771190,1.684479
0,SOEHWGF12A6D4F8B2B,249,1.743513,1.680140


💡 **Observations and Insights:**

*    The larger the frequency the smaller gap between predicted and corrected ratings
*    There are not significant outliers between predicted and corrected ratings



### Model Based Collaborative Filtering - Matrix Factorization

Model-based Collaborative Filtering is a **personalized recommendation system**, the recommendations are based on the past behavior of the user and it is not dependent on any additional information. We use **latent features** to find recommendations for each user.

In [ ]:
# Build baseline model using svd
# Using SVD matrix factorization
svd = SVD(random_state=1)

# Training the algorithm on the trainset
svd.fit(trainset)

# Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_recall_at_k(svd)

RMSE: 1.0252
Precision:  0.41
Recall:  0.633
F_1 score:  0.498


In [ ]:
# Making prediction for user (with user_id 6958) to song (with song_id 1671), take r_ui = 2
svd.predict("6958", 1671, r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=1671, r_ui=2, est=1.6989607635206787, details={'was_impossible': False})

In [ ]:
# Making a prediction for the user who has not listened to the song (song_id 3232)
svd.predict("6958", 3232, verbose = True)

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=3232, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

#### Improving matrix factorization based recommendation system by tuning its hyperparameters

In [ ]:
# Set the parameter space to tune
param_grid = {'n_epochs': [10, 20, 30], 'lr_all': [0.001, 0.005, 0.01],
              'reg_all': [0.2, 0.4, 0.6]}

# Performe 3-fold grid-search cross-validation
gs = GridSearchCV(SVD, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html).

In [ ]:
# Building the optimized SVD model using optimal hyperparameters
svd_optimized = SVD(n_epochs= 30, lr_all= 0.01, reg_all= 0.2, random_state=1)

# Train the algorithm on the trainset
svd_optimized=svd_optimized.fit(trainset)

# Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_recall_at_k(svd_optimized)


RMSE: 1.0141
Precision:  0.415
Recall:  0.635
F_1 score:  0.502


💡 **Observations and Insights:**

*   Root mean square error (RMSE) is 0.___
*   The precision is ___ meaning that out of the recommended items ___ were relevant to the user. 
*   The recall is ___ meaning that out of the relevant products ___ products are recommended.
*  With an F_1 score of ___ which is representative of both recall and precision.

In [ ]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 1671
svd_optimized.predict('6958', 1671, verbose=True)

user: 6958       item: 1671       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=1671, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

In [ ]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline rating
svd_optimized.predict("6958", 3232, verbose=True)

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=3232, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

💡 **Observations and Insights:**

*    Estimated rating prediction is the same for both song_ids

In [ ]:
# Getting top 5 recommendations for user_id 6958 using "svd_optimized" algorithm
svd_recommendations = get_recommendations(df_final, 6958, 5, svd_optimized )

In [ ]:
# Ranking songs based on above recommendations
ranking_songs(svd_recommendations, final_play)

,song_id,play_freq,predicted_ratings,corrected_ratings
2,SOSJSSU12A6D4F8F41,107,2.601899,2.505225
1,SOOGNOZ12AAF3B2936,108,2.108728,2.012502
4,SOVIZNF12AF72A710A,96,2.014091,1.912029
0,SOZVVRE12A8C143150,150,1.940115,1.858465
3,SOQGSUC12A8C13B66D,102,1.952493,1.853478


💡 **Observations and Insights:**

*   The closest predicted rating compared to corrected rating song_id is SOZVVRE12A8C143150
*   The larger the frequency the smaller gap between predicted and corrected ratings
*   It would be interesting to see if we were able to calculate a confidence interval or score on the predicted ratings



### Cluster Based Recommendation System

In **clustering-based recommendation systems**, we explore the **similarities and differences** in people's tastes in songs based on how they rate different songs. We cluster similar users together and recommend songs to a user based on play_counts from other users in the same cluster.

In [ ]:
# Make baseline clustering model
# Using Co-Clustering algorithm
clust_baseline = CoClustering(random_state = 1)

# Training the algorithm on the trainset
clust_baseline.fit(trainset)

# Let us compute precision@k and recall@k with k = 10
precision_recall_at_k(clust_baseline)

RMSE: 1.0487
Precision:  0.397
Recall:  0.582
F_1 score:  0.472


In [ ]:
# Making prediction for user_id 6958 and song_id 1671
clust_baseline.predict('6958', 1671, verbose = True)

user: 6958       item: 1671       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=1671, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

In [ ]:
# Making prediction for user (userid 6958) for a song(song_id 3232) not heard by the user
clust_baseline.predict('6958', 3232, verbose = True)

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid='6958', iid=3232, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

#### Improving clustering-based recommendation system by tuning its hyper-parameters

In [ ]:
# Set the parameter space to tune
param_grid = {'n_cltr_u': [5, 6, 7, 8], 'n_cltr_i': [5, 6, 7, 8], 'n_epochs': [10, 20, 30]}

# Performing 3-fold gridsearch cross validation
gs = GridSearchCV(CoClustering, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/co_clustering.html).

In [ ]:
# Train the tuned Coclustering algorithm
# Using tuned Coclustering algorithm
clust_tuned = CoClustering(n_cltr_u = 5,n_cltr_i = 5, n_epochs = 10, random_state = 1)

# Training the algorithm on the trainset
clust_tuned.fit(trainset)

# Let us compute precision@k and recall@k with k = 10
precision_recall_at_k(clust_tuned)


RMSE: 1.0654
Precision:  0.394
Recall:  0.566
F_1 score:  0.465


💡 **Observations and Insights:**

*   Root mean square error (RMSE) is 1.0654
*   The precision is 0.394 meaning that out of the recommended items 0.394 were relevant to the user. 
*   The recall is 0.566 meaning that out of the relevant products 0.566 products are recommended.
*  With an F_1 score of 0.465 which is representative of both recall and precision.

In [ ]:
# Using co_clustering_optimized model to recommend for userId 6958 and song_id 1671
clust_tuned.predict(6958, 1671, verbose = True)

user: 6958       item: 1671       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid=6958, iid=1671, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

In [ ]:
# Use Co_clustering based optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline rating
clust_tuned.predict(6958, 3232, verbose = True) 

user: 6958       item: 3232       r_ui = None   est = 1.70   {'was_impossible': False}


Prediction(uid=6958, iid=3232, r_ui=None, est=1.6989607635206787, details={'was_impossible': False})

💡 **Observations and Insights:**

*    Estimated rating prediction is the same for both song_ids

#### Implementing the recommendation algorithm based on optimized CoClustering model

In [ ]:
def get_recommendations(data, user_id, top_n, algo):
    
    # Creating an empty list to store the recommended restaurants ids
    recommendations = []
    
    # Creating an user item interactions matrix 
    user_item_interactions_matrix = data.pivot_table(index = 'user_id', columns = 'song_id', values = 'play_count')
    
    # Extracting those restaurants ids which the user_id has not interacted yet
    non_interacted_restaurants = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()
    
    # Looping through each of the restaurants ids which user_id has not interacted yet
    for item_id in non_interacted_restaurants:
        
        # Predicting the ratings for those non interacted restaurants ids by this user
        est = algo.predict(user_id, item_id).est
        
        # Appending the predicted ratings
        recommendations.append((item_id, est))

    # Sorting the predicted ratings in descending order
    recommendations.sort(key = lambda x: x[1], reverse = True)

    # Returing top n highest predicted rating restaurants for this user
    return recommendations[:top_n]

In [ ]:
# Getting top 5 recommendations for user_id 6958 using "Co-clustering based optimized" algorithm
clustering_recommendations = get_recommendations(df_final, 6958, 5, clust_tuned)

### Correcting the play_count and Ranking the above songs

In [ ]:
# Ranking songs based on the above recommendations
ranking_songs(clustering_recommendations, final_play)

,song_id,play_freq,predicted_ratings,corrected_ratings
4,SOSJSSU12A6D4F8F41,107,3.711503,3.614829
3,SOOGNOZ12AAF3B2936,108,2.903883,2.807658
0,SORJICW12A8C13640D,169,2.691043,2.614120
1,SOBRHVR12A8C133F35,151,2.606354,2.524975
2,SOVWBYM12A6D4F8A22,123,2.582807,2.492640


💡 **Observations and Insights:**


*    Play_freq average is less than item to item play frequency songs
*    The highest predicted rating is 3.7 which is close to the corrected rating of 3.6

### Content Based Recommendation Systems

**Think About It:** So far we have only used the play_count of songs to find recommendations but we have other information/features on songs as well. Can we take those song features into account?

In [ ]:
df_small = df_final

In [ ]:
# Concatenate the "title", "release", "artist_name" columns to create a different column named "text"
df_small['text'] = df_small['title'] + ' ' + df_small['release'] + ' ' + df_small['artist_name']

In [ ]:
# Select the columns 'user_id', 'song_id', 'play_count', 'title', 'text' from df_small data
df_small[['user_id', 'song_id', 'play_count', 'title', 'text']]

# Drop the duplicates from the title column
df_small = df_small.drop_duplicates(subset = ['title'])

# Set the title column as the index
df_small = df_small.set_index('title')

# See the first 5 records of the df_small dataset
df_small.head()


,user_id,song_id,play_count,release,artist_name,year,business_id,text
title,,,,,,,,
Daisy And Prudence,6958,SOBDVAK12AC90759A2,1,Distillation,Erin McKeown,2000,447,Daisy And Prudence Distillation Erin McKeown
The Ballad of Michael Valentine,6958,SOBIMTY12A6D4F931F,1,Sawdust,The Killers,2004,512,The Ballad of Michael Valentine Sawdust The Ki...
I Stand Corrected (Album),6958,SOBKRVG12A8C133269,1,Vampire Weekend,Vampire Weekend,2007,549,I Stand Corrected (Album) Vampire Weekend Vamp...
They Might Follow You,6958,SOBUBLL12A58A795A8,1,Tiny Vipers,Tiny Vipers,2007,703,They Might Follow You Tiny Vipers Tiny Vipers
Monkey Man,6958,SOBVKFF12A8C137A79,1,You Know I'm No Good,Amy Winehouse,2007,719,Monkey Man You Know I'm No Good Amy Winehouse


In [ ]:
# Create the series of indices from the data
indices = pd.Series(df_small.index)

indices[ : 5]

0                 Daisy And Prudence
1    The Ballad of Michael Valentine
2          I Stand Corrected (Album)
3              They Might Follow You
4                         Monkey Man
Name: title, dtype: object

In [ ]:
# Importing necessary packages to work with text data
import nltk
nltk.download('omw-1.4')

# Download punkt library
nltk.download("punkt")

# Download stopwords library
nltk.download("stopwords")

# Download wordnet 
nltk.download("wordnet")

# Import regular expression
import re

# Import word_tokenizer
from nltk import word_tokenize

# Import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer

# Import stopwords
from nltk.corpus import stopwords

# Import CountVectorizer and TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


We will create a **function to pre-process the text data:**

In [ ]:
# Function to tokenize the text
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z]"," ", text.lower())
    
    tokens = word_tokenize(text)
    
    words = [word for word in tokens if word not in stopwords.words('english')]  # Use stopwords of english
    
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]

    return text_lems


In [ ]:
# Create tfidf vectorizer 
tfidf = TfidfVectorizer(tokenizer = tokenize)

# Fit_transfrom the above vectorizer on the text column and then convert the output into an array
song_tfidf = tfidf.fit_transform(df_small['text'].values).toarray()

In [ ]:
# Compute the cosine similarity for the tfidf above output
similar_songs = cosine_similarity(song_tfidf, song_tfidf)

similar_songs

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

 Finally, let's create a function to find most similar songs to recommend for a given song.

In [ ]:
# Function that takes in song title as input and returns the top 10 recommended songs
def recommendations(title, similar_songs):
    
    recommended_songs = []
    
    # Getting the index of the song that matches the title
    idx = indices[indices == title].index[0]

    # Creating a Series with the similarity scores in descending order
    score_series = pd.Series(similar_songs[idx]).sort_values(ascending = False)

    # Getting the indexes of the 10 most similar songs
    top_10_indexes = list(score_series.iloc[1 : 11].index)
    print(top_10_indexes)
    
    # Populating the list with the titles of the best 10 matching songs
    for i in top_10_indexes:
        recommended_songs.append(list(df_small.index)[i])
        
    return recommended_songs

Recommending 10 songs similar to Learn to Fly

In [ ]:
# Make the recommendation for the song with title 'Learn To Fly'
recommendations('Learn To Fly', similar_songs)

[509, 234, 423, 345, 394, 370, 371, 372, 373, 375]


['Everlong',
 'The Pretender',
 'Nothing Better (Album)',
 'From Left To Right',
 'Lifespan Of A Fly',
 'Under The Gun',
 'I Need A Dollar',
 'Feel The Love',
 'All The Pretty Faces',
 'Bones']

**Observations and Insights:**

## **Conclusion and Recommendations:** 

Refined insights: Spotify Recommendations Module 2

  |           | User-Based Filtering                   | Item-Based Filtering                   | Matrix Factorization                   | Cluster-Based Model                   |
| --------- | -------------------------------------- | -------------------------------------- | -------------------------------------- | ------------------------------------- |
| RMSE      | 1.0626                                 | 1.0394                                 | 1.0252                                 | 1.0487                                |
| Precision | 0.413                                  | 0.307                                  | 0.41                                   | 0.397                                 |
| Recall    | 0.608                                  | 0.562                                  | 0.633                                  | 0.582                                 |
| F_1 score | 0.492                                  | 0.397                                  | 0.498                                  | 0.472                                 |
|           | User-Based Filtering w/Hyperparameters | Item-Based Filtering w/Hyperparameters | Matrix Factorization w/Hyperparameters | Cluster-Based Model w/Hyperparameters |
| RMSE      | 1.0534                                 | 1.0423                                 | **1.0141**                             | 1.0654                                |
| Precision | 0.414                                  | 0.34                                   | **0.415**                              | 0.394                                 |
| Recall    | **0.703**                              | 0.563                                  | 0.635                                  | 0.566                                 |
| F_1 score | 0.521                                  | 0.424                                  | 0.502                                  | 0.465                                 |


**What are the most meaningful insights from the data relevant to the
problem?**

-   After completing two individual recommendation systems I realized
    that the most appropriate model depends on the data that is
    provided. No one model works for every recommendation system.
    Various techniques range from a simple model to a more complex and
    layered one. The most appropriate model is determined through what
    can provide the most accurate prediction rating and this varies
    based on the business ask.

-   User to User filtering models with and without hyperparameters have
    the largest discrepancies between F_1 score values. This indicates
    that tuning the model can make a significant impact on your model
    performance. Matrix factorization considers the user to item
    filtering, this allows for a more personalized recommendation system
    that is strictly based on past behaviors of the user. I am surprised
    this does not have a higher F_1 score but it was the second highest.
    Additionally, I would like to explore the models with different
    parameters and test sizes. Will increasing test size and decreasing
    parameters for specific models drastically change the result we are
    already seeing?

**Comparison of various techniques and their relative performance:**

**How do different techniques perform?**

-   Item Based Filter was the least efficient model, realistically this
    makes sense. Item to Item modeling was filtered on song_id and
    recommended similar songs according to the data. This is difficult
    to conduct without grouping/clustering into something such as
    genres. Users listen to varying types of songs and a singular song
    may fall into two opposite categories. To make the item filtering
    more efficient we would want to follow a cluster-based modeling on a
    grouping such as a genre. Item-to-item-based filtering with and
    without tuning did not lead to promising results. Per the reference
    chart above it was the worst-performing model based on the F_1
    score. Matrix factorization led to the second-highest scores
    compared to other models.

**Which one is performing better?­­­**

-   User to User w/Hyperparameters has indicated the highest F_1 score
    compared to all the filtering models with a 0.521 score. Tuning
    improved the F_1 score the most compared to other models and
    contains the highest recall rate.

**Is there scope to improve the performance further?**

-   To improve the performance there are additional data points that I
    would like to help organize the data. Additional data may include
    genre, mood tags, and producer credits. Adding these aspects into
    models will help clarify a more direct recommendation without adding
    too much noise. Mood tags can help group songs into themes and
    finding out at what times of the day a user listens to a specific
    mood can enhance recommendations. Producers tend to write a similar
    style of songs thus producer credits can recommend more songs not by
    artist popularity but by producer popularity.

**Proposal for the final solution design:**

**What model do you propose to be adopted? Why is this the best solution
to adopt?**

-   I propose User-Based Filtering w/Hyperparameters because of the F_1
    score, this is a major indicator of how accurate the predictions for
    the said model will be. After testing this with r_ui I was able to
    determine that the prediction was closest to the real rating by
    0.30. As previously mentioned, adding items such as genre, mood
    tags, and producer credits can enhance the product and are easy to
    implement for an item-based model. Users tend to not explore outside
    of their initial preferred artists or genres which could lead to a
    linear experience. Spotify prides itself on being able to conduct a
    musical experience through its recommendations. When focusing on
    items such as song_id it is not based on previous individual user
    insights but more tailored towards similar songs recently listened
    to or popularly played. This allows for a more precise prediction.



